In [1]:
from src.data_preprocessor import DataProcessor
from data_configs.configs import *
from models.decision_tree import DecisionTree, DecisionTreeNode
from models.null_model import NullModelClassification, NullModelRegression
from src.cross_validation import CrossValidation
from src.evaluation import Evaluation
import numpy as np

config = albalone_config
decision_tree = DecisionTree(config=config)
data_processor = DataProcessor(config=config)
cross_validator = CrossValidation(config=config)

### Data Load and Preprocessing ###

In [2]:
raw_data = data_processor.load_data()

data_1 = data_processor.impute_missing_values(raw_data)

In [3]:
data_1.describe()

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
count,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000
mean,0.523992,0.407881,0.139516,0.828742,0.359367,0.180594,0.238831,9.933684
std,0.120093,0.099240,0.041827,0.490389,0.221963,0.109614,0.139203,3.224169
min,0.075000,0.055000,0.000000,0.002000,0.001000,0.000500,0.001500,1.000000
25%,0.450000,0.350000,0.115000,0.441500,0.186000,0.093500,0.130000,8.000000
50%,0.545000,0.425000,0.140000,0.799500,0.336000,0.171000,0.234000,9.000000
75%,0.615000,0.480000,0.165000,1.153000,0.502000,0.253000,0.329000,11.000000
max,0.815000,0.650000,1.130000,2.825500,1.488000,0.760000,1.005000,29.000000


In [4]:
data_train, data_val = cross_validator.random_partition(data_1, random_state=42)

## Decision Tree Performance ##

In [5]:
# Initialize lists to store scores for decision tree, pruned decision tree, and null model
dt_scores = {'mse': [], 'mae': [], 'r2': [], 'pearson_correlation': []}
pruned_dt_scores = {'mse': [], 'mae': [], 'r2': [], 'pearson_correlation': []}
null_model_scores = {'mse': [], 'mae': []}  

for i, (train_set, test_set) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):
    train_data = train_set.drop(columns=config['target_column'])
    train_target = train_set[config['target_column']]
    test_features = test_set.drop(columns=config['target_column'])
    test_true_vals = test_set[config['target_column']]

    # Decision Tree Model
    decision_tree = DecisionTree(config)
    decision_tree.root = decision_tree.build_regression_tree(train_data, train_target)
    predictions = decision_tree.predict(test_features)

    # Calculate and store decision tree scores
    scores = Evaluation.calculate_regression_scores(test_true_vals, predictions)
    for key in dt_scores:
        dt_scores[key].append(scores[key])

    # Pruning the Decision Tree
    decision_tree.prune(decision_tree.root, data_val)  # Ensure data_val is correctly defined as your validation set
    pruned_predictions = decision_tree.predict(test_features)

    # Calculate and store pruned decision tree scores
    pruned_scores = Evaluation.calculate_regression_scores(test_true_vals, pruned_predictions)
    for key in pruned_dt_scores:
        pruned_dt_scores[key].append(pruned_scores[key])

    # Null Model
    null_model = NullModelRegression(config=config)
    null_model_prediction = null_model.naive_regression(test_set)

    # Calculate and store null model scores (excluding R2 and Pearson)
    null_model_mse = Evaluation.mean_squared_error(test_true_vals, null_model_prediction)
    null_model_mae = Evaluation.mean_absolute_error(test_true_vals, null_model_prediction)
    null_model_scores['mse'].append(null_model_mse)
    null_model_scores['mae'].append(null_model_mae)

# Calculate average scores for each model
average_dt_scores = {metric: np.mean(values) for metric, values in dt_scores.items()}
average_pruned_dt_scores = {metric: np.mean(values) for metric, values in pruned_dt_scores.items()}
average_null_model_scores = {metric: np.mean(values) for metric, values in null_model_scores.items()}

# Print average scores
print("Average Decision Tree Scores:")
for metric, avg_score in average_dt_scores.items():
    print(f"{metric}: {avg_score}")

print("\nAverage Pruned Decision Tree Scores:")
for metric, avg_score in average_pruned_dt_scores.items():
    print(f"{metric}: {avg_score}")

print("\nAverage Null Model Scores:")
for metric, avg_score in average_null_model_scores.items():
    print(f"{metric}: {avg_score}")


Average Decision Tree Scores:
mse: 8.724732408065442
mae: 2.0888224558471125
r2: 0.15099786538982946
pearson_correlation: 0.5753762808825862

Average Pruned Decision Tree Scores:
mse: 7.998683204768071
mae: 1.9620462492215605
r2: 0.22163514659253192
pearson_correlation: 0.5354810403434506

Average Null Model Scores:
mse: 10.280610937065877
mae: 2.357776660660252
